# Installing Dependencies

In [ ]:
!pip install tensorflow;
!pip install keras;
!pip install opencv-python;
!pip install matplotlib;
!pip install pandas;
!pip install numpy;
!pip install scikit-learn;
!pip install seaborn;
!pip install matplotlib;
!pip install transformers;
!pip install tf-keras-vis --quiet

In [ ]:
!pip install timm

In [ ]:
!pip install segmentation-models tensorflow_addons
!pip install git+https://github.com/qubvel/segmentation_models

# Loading Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Dataset Preparation and Sample Image Display

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

data_folder = '/content/drive/MyDrive/data'
dataset_path = '/content/dataset'


if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

subfolders = [f for f in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, f))]
for folder in subfolders:
    source_folder = os.path.join(data_folder, folder)
    destination_folder = os.path.join(dataset_path, folder)

    if not os.path.exists(destination_folder):
        shutil.copytree(source_folder, destination_folder)
    else:
        print(f"Folder {folder} already exists in {dataset_path}. Skipping.")
    print(f"Copied {folder} to {destination_folder}")


all_image_paths = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            all_image_paths.append(os.path.join(root, file))

if all_image_paths:
    random_image_path = random.choice(all_image_paths)
    img = mpimg.imread(random_image_path)
    plt.imshow(img)
    plt.title(f"Random Image from {random_image_path}")
    plt.axis('off')
    plt.show()
else:
    print("No images found in the dataset.")

#  Dataset Image Count by Class

In [ ]:
category_counts = {}
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        num_images = len([f for f in os.listdir(category_path) if f.endswith(('.png', '.jpg', '.jpeg'))])
        category_counts[category] = num_images

# Print the results:
for category, count in category_counts.items():
    print(f"Category '{category}': {count} images")

# Count of Total Images

In [ ]:
import os

dataset_path = '/content/dataset'
total_images = 0
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
               total_images += 1

print(f"Total number of images in the dataset: {total_images}")

#Data Loading, Preprocessing, and Splitting Setup

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import cv2


data_folder = '/content/drive/MyDrive/data'
dataset_path = '/content/dataset'

IMG_SIZE = 224

image_paths = []
labels = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(root, file))
            labels.append(os.path.basename(os.path.dirname(os.path.join(root, file))))



train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)


val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42
)


def load_and_resize(path):
    img = mpimg.imread(path)

    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

train_images = np.array([load_and_resize(path) for path in train_paths])
val_images = np.array([load_and_resize(path) for path in val_paths])
test_images = np.array([load_and_resize(path) for path in test_paths])


from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)


from tensorflow.keras.utils import to_categorical
train_labels_categorical = to_categorical(train_labels_encoded)
val_labels_categorical = to_categorical(val_labels_encoded)
test_labels_categorical = to_categorical(test_labels_encoded)

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_datagen.flow(
    train_images,
    train_labels_categorical,
    batch_size=32,
    shuffle=False
)


val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_data = val_datagen.flow(
    val_images,
    val_labels_categorical,
    batch_size=32,

)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(
    test_images,
    test_labels_categorical,
    batch_size=32,
    shuffle=False

)

In [ ]:
from collections import Counter
train_class_counts = Counter(train_labels)
print("Number of images per class in Training Set:")
for label, count in train_class_counts.items():
    print(f"Category '{label}': {count} images")

val_class_counts = Counter(val_labels)
print("\nNumber of images per class in Validation Set:")
for label, count in val_class_counts.items():
    print(f"Category '{label}': {count} images")

test_class_counts = Counter(test_labels)
print("\nNumber of images per class in Testing Set:")
for label, count in test_class_counts.items():
    print(f"Category '{label}': {count} images")

#ResNet50

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import numpy as np
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


IMG_SIZE = 224

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))




x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
)

# Evaluate the model on the test set
print("\nEvaluating the model on the test set...")
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded
report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)

In [ ]:
model.save('my_trained_model.h5')
print("Model saved successfully!")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras

model = keras.models.load_model('my_trained_model.h5')


random_index = np.random.randint(0, len(test_paths))
image_path = test_paths[random_index]
true_label = test_labels[random_index]


img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = keras.applications.resnet50.preprocess_input(img_array)


prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


plt.imshow(img)
plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
plt.axis('off')
plt.show()

# ResNet152

In [ ]:
from tensorflow.keras.applications import ResNet152


base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


for layer in base_model.layers[:-50]:
    layer.trainable = False
for layer in base_model.layers[-50:]:
    layer.trainable = True


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
    callbacks=[lr_reduction] # Add callbacks here
)


loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded
report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)

model.save('resnet152_trained_model.h5')
print("ResNet152 model saved successfully!")


model = keras.models.load_model('resnet152_trained_model.h5')


random_index = np.random.randint(0, len(test_paths))
image_path = test_paths[random_index]
true_label = test_labels[random_index]


img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = keras.applications.resnet_v2.preprocess_input(img_array)


prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


plt.imshow(img)
plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
plt.axis('off')
plt.show()


# ResNet101

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet101, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import cv2


IMG_SIZE = 224
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
)


loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


from sklearn.metrics import classification_report
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded

report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)


model.save('my_trained_model_resnet101.h5')
print("Model saved successfully!")

In [ ]:

from sklearn.metrics import classification_report
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded

report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)


model.save('my_trained_model_resnet101.h5')
print("Model saved successfully!")

#EfficientNetV2B0

In [ ]:
!pip install -U efficientnet_v2
from tensorflow.keras.applications import EfficientNetV2B0


IMG_SIZE = 224

base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)


num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=7, factor=0.5, verbose=1)


epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
    callbacks=[early_stopping, lr_reduction]  # Include the callbacks
)


print("\nEvaluating the EfficientNetV2B0 model on the test set...")
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded


report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)


model.save('efficientnetv2b0_trained_model.h5')
print("EfficientNetV2B0 model saved successfully!")


random_index = np.random.randint(0, len(test_paths))
image_path = test_paths[random_index]
true_label = test_labels[random_index]


img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = keras.applications.efficientnet_v2.preprocess_input(img_array)


prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


plt.imshow(img)
plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
plt.axis('off')
plt.show()

#EfficientNetV2S

In [ ]:
!pip install -U efficientnet_v2
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras


IMG_SIZE = 224


base_model = EfficientNetV2S(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


num_classes = 4
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,

)


print("\nEvaluating the EfficientNetV2S model on the test set...")
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded


report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)


model.save('efficientnetv2s_trained_model.h5')
print("EfficientNetV2S model saved successfully!")

random_index = np.random.randint(0, len(test_paths))
image_path = test_paths[random_index]
true_label = test_labels[random_index]


img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = keras.applications.efficientnet_v2.preprocess_input(img_array)

prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

plt.imshow(img)
plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
plt.axis('off')
plt.show()

#EfficientNetB7

In [ ]:
#!pip install -U efficientnet
from tensorflow.keras.applications import EfficientNetB7

IMG_SIZE = 224

base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))


for layer in base_model.layers[:-50]:
    layer.trainable = False
for layer in base_model.layers[-50:]:
    layer.trainable = True


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)


epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
    callbacks=[early_stopping, lr_reduction]
)


print("\nEvaluating the EfficientNetB7 model on the test set...")
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded


report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)

model.save('efficientnetb7_trained_model.h5')
print("EfficientNetB7 model saved successfully!")

random_index = np.random.randint(0, len(test_paths))
image_path = test_paths[random_index]
true_label = test_labels[random_index]


img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = keras.applications.efficientnet.preprocess_input(img_array)


prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

plt.imshow(img)
plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
plt.axis('off')
plt.show()

# MobileNetV2

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.callbacks import EarlyStopping

IMG_SIZE = 224
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
)

In [ ]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


from sklearn.metrics import classification_report
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded

report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)

#VGG16

In [ ]:

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

IMG_SIZE = 224


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
)


print("\nEvaluating the VGG16 model on the test set...")
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded


report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)


model.save('vgg16_trained_model.h5')
print("VGG16 model saved successfully!")

random_index = np.random.randint(0, len(test_paths))
image_path = test_paths[random_index]
true_label = test_labels[random_index]

img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

plt.imshow(img)
plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
plt.axis('off')
plt.show()

#VGG19

In [ ]:

from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input

IMG_SIZE = 224


base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 50
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data,
)

print("\nEvaluating the VGG16 model on the test set...")
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_labels_encoded

report = classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_)
print(report)

model.save('vgg16_trained_model.h5')
print("VGG16 model saved successfully!")

random_index = np.random.randint(0, len(test_paths))
image_path = test_paths[random_index]
true_label = test_labels[random_index]


img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)
predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


plt.imshow(img)
plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
plt.axis('off')
plt.show()

# Vision Transformer

In [ ]:
!pip install tqdm

from tqdm import tqdm
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class_names = ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'no_tumor']


data_path = '/content/dataset'


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder(data_path, transform=transform)


total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = int(0.10 * total_size)
test_size = total_size - train_size - val_size


train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size]
)


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


model = timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=len(class_names))
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)


epochs = 50

best_val_acc = 0.0
best_model_state = None

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0


    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    train_loss = running_loss / len(train_loader)


    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss

In [ ]:

model.eval()

train_correct = 0
train_total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

train_accuracy = 100 * train_correct / train_total
print(f"Training Accuracy: {train_accuracy:.2f}%")


y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.cpu().numpy())

test_accuracy = 100 * np.sum(np.array(y_true) == np.array(y_pred)) / len(y_true)
print(f"Testing Accuracy: {test_accuracy:.2f}%")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))



# X-RVV

In [ ]:
import tensorflow as tf
from keras.applications import ResNet50, VGG19
from keras.layers import Input, Dense, Dropout, concatenate, GlobalAveragePooling2D, Resizing, Rescaling
from keras.models import Model
from keras.optimizers import Adam
from transformers import TFViTModel
import segmentation_models as sm


vit_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

IMG_SIZE = 224
NUM_CLASSES = 4


input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input_layer')


resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)
resnet_out = GlobalAveragePooling2D()(resnet.output)


vgg = VGG19(include_top=False, weights='imagenet', input_tensor=input_tensor)
vgg_out = GlobalAveragePooling2D()(vgg.output)


vit_input = Resizing(224, 224)(input_tensor)
vit_input = Rescaling(1./255)(vit_input)

class TransposeLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.transpose(inputs, perm=[0, 3, 1, 2])

vit_input_transposed = TransposeLayer()(vit_input)

class ViTBranch(tf.keras.layers.Layer):
    def call(self, inputs, training=False):
        outputs = vit_model({'pixel_values': inputs})
        return outputs.last_hidden_state[:, 0, :]

vit_out = ViTBranch()(vit_input_transposed)


merged = concatenate([resnet_out, vgg_out, vit_out])
x = Dense(1024, activation='relu')(merged)
x = Dropout(0.6)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
classification_output = Dense(NUM_CLASSES, activation='softmax', name='classification_output')(x)


segmentation_output = sm.Unet(
    backbone_name='resnet34',
    encoder_weights='imagenet',
    classes=1,
    activation='sigmoid'
)(input_tensor)
segmentation_output._name = 'segmentation_output'


model = Model(inputs=input_tensor, outputs=[classification_output, segmentation_output], name="functional_37")


classification_model = Model(inputs=model.input, outputs=model.get_layer('classification_output').output, name="functional_37")


classification_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

classification_model.summary()


In [ ]:

history = classification_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[
        keras.callbacks.ModelCheckpoint('best_hybrid_model.keras', save_best_only=True)
    ]
)


In [ ]:
import keras
import tensorflow as tf
from keras.models import load_model

classification_model = load_model('best_hybrid_model.keras', custom_objects={'TransposeLayer': TransposeLayer, 'ViTBranch': ViTBranch})


In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = classification_model.predict(images)
    y_true.extend(tf.argmax(labels, axis=1).numpy())
    y_pred.extend(tf.argmax(preds, axis=1).numpy())

y_true = tf.convert_to_tensor(y_true)
y_pred = tf.convert_to_tensor(y_pred)


In [ ]:
test_loss, test_accuracy = classification_model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
from sklearn.metrics import classification_report

class_names = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

report = classification_report(y_true, y_pred, target_names=class_names, digits=4)
print(report)

In [ ]:


from sklearn.metrics import roc_auc_score
import numpy as np

y_true_auc = []
y_prob = []


for images, labels in test_ds:

    probs = classification_model.predict(images, verbose=0)
    y_true_auc.extend(tf.argmax(labels, axis=1).numpy())
    y_prob.extend(probs)

y_true_auc = np.array(y_true_auc)
y_prob = np.array(y_prob)


auc_score_ovr = roc_auc_score(y_true_auc, y_prob, multi_class='ovr')


auc_score_macro = roc_auc_score(y_true_auc, y_prob, multi_class='ovr', average='macro')


auc_score_weighted = roc_auc_score(y_true_auc, y_prob, multi_class='ovr', average='weighted')


print(f"\nAUC Score (One-vs-Rest): {auc_score_ovr:.4f}")
print(f"AUC Score (Macro Average): {auc_score_macro:.4f}")
print(f"AUC Score (Weighted Average): {auc_score_weighted:.4f}")


for i, class_name in enumerate(class_names):

    y_true_class = (y_true_auc == i).astype(int)

    y_prob_class = y_prob[:, i]
    auc_class = roc_auc_score(y_true_class, y_prob_class)
    print(f"AUC for class '{class_name}': {auc_class:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix - Test Set")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np


fpr = dict()
tpr = dict()
roc_auc = dict()

n_classes = len(class_names)

for i in range(n_classes):

    y_true_bin = (y_true_auc == i).astype(int)


    fpr[i], tpr[i], _ = roc_curve(y_true_bin, y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves for each class
plt.figure(figsize=(8, 6))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--') # plot the diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Per Class')
plt.legend(loc="lower right")
plt.show()

# Plot separate ROC curves for each class
for i in range(n_classes):
    plt.figure(figsize=(6, 5))
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for Class: {class_names[i]}')
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
!pip install tf-keras-vis --quiet


#Data Augmentation

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


augmented_train_dir = '/content/augmented_data'
os.makedirs(augmented_train_dir, exist_ok=True)


augmentations_per_image = 3

print(f"Augmenting {len(train_paths)} training images. Generating {augmentations_per_image} augmented images per original image.")

for i, img_path in enumerate(train_paths):
    try:
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)


        category = os.path.basename(os.path.dirname(img_path))


        category_folder = os.path.join(augmented_train_dir, category)
        os.makedirs(category_folder, exist_ok=True)


        prefix = f"{os.path.splitext(os.path.basename(img_path))[0]}_aug"


        gen_count = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=category_folder, save_prefix=prefix, save_format='jpeg'):
            gen_count += 1
            if gen_count >= augmentations_per_image:
                break

    except Exception as e:
        print(f"Error augmenting image {img_path}: {e}")

print("Augmentation process complete.")



# Augmented Data Splitting

In [ ]:
from keras.utils import image_dataset_from_directory

train_ds = image_dataset_from_directory(
    '/content/augmented_data',
    labels='inferred',
    label_mode='categorical',  # for softmax output
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = image_dataset_from_directory(
    '/content/augmented_data',
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)


train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from collections import Counter

train_class_counts = Counter(train_labels)
print("Training set class distribution:")
for class_name, count in train_class_counts.items():
    print(f"{class_name}: {count} images")

print("\n" + "-"*40 + "\n")


val_class_counts = Counter(val_labels)
print("Validation set class distribution:")
for class_name, count in val_class_counts.items():
    print(f"{class_name}: {count} images")


#RESNET50 + DA

In [ ]:

print("Unfreezing layers for fine-tuning...")
for layer in base_model.layers:
    layer.trainable = True


num_layers_to_unfreeze = 50

for layer in base_model.layers[:-num_layers_to_unfreeze]:
    layer.trainable = False
print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")


from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model recompiled for fine-tuning with a lower learning rate.")
model.summary()


epochs_finetune = 50


from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau # Import callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)


print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")
history_finetune = model.fit(
    augmented_train_data,
    epochs=epochs_finetune,
    validation_data=val_data,
    #callbacks=[early_stopping, lr_reduction]
)


print("\nEvaluating the fine-tuned ResNet50 model on the test set...")
loss_finetune, accuracy_finetune = model.evaluate(test_data)
print(f"Fine-tuned Test Loss: {loss_finetune}")
print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


from sklearn.metrics import classification_report
print("\nClassification Report for Fine-tuned ResNet50:")


y_true = test_labels_encoded


y_pred_finetune = model.predict(test_data)
y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)
report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
print(report_finetune)


model.save('resnet50_finetuned_model.h5')
print("Fine-tuned ResNet50 model saved successfully!")


plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_finetune.history['accuracy'])
plt.plot(history_finetune.history['val_accuracy'])
plt.title('Fine-tuned Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')


plt.subplot(1, 2, 2)
plt.plot(history_finetune.history['loss'])
plt.plot(history_finetune.history['val_loss'])
plt.title('Fine-tuned Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

#ResNet152 + DA

In [ ]:

from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras



IMG_SIZE = 224


base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


print("Freezing base ResNet152 layers...")
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)


epochs_initial = 20
print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")
history_initial = model.fit(
    train_ds,
    epochs=epochs_initial,
    validation_data=val_ds,
    #callbacks=[early_stopping, lr_reduction]
)


print("\nUnfreezing layers for fine-tuning...")


for layer in base_model.layers:
    layer.trainable = True


num_layers_to_unfreeze = 100


for layer in base_model.layers[:-num_layers_to_unfreeze]:
    layer.trainable = False
print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")




optimizer_finetune = Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model recompiled for fine-tuning with a very low learning rate.")
model.summary()

epochs_finetune = 50
print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")


history_finetune = model.fit(
    augmented_train_data,
    epochs=epochs_finetune,
    validation_data=val_data,
    #callbacks=[early_stopping, lr_reduction]
)


print("\nEvaluating the fine-tuned ResNet152 model on the test set...")
loss_finetune, accuracy_finetune = model.evaluate(test_data)
print(f"Fine-tuned Test Loss: {loss_finetune}")
print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


print("\nClassification Report for Fine-tuned ResNet152:")


y_true = test_labels_encoded


y_pred_finetune = model.predict(test_data)
y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)

report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
print(report_finetune)


model.save('resnet152_finetuned_model.h5')
print("Fine-tuned ResNet152 model saved successfully!")


if 'test_paths' in globals() and 'test_labels' in globals():
    random_index = np.random.randint(0, len(test_paths))
    image_path = test_paths[random_index]
    true_label = test_labels[random_index]


    img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)



    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


    plt.imshow(img)
    plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    plt.axis('off')
    plt.show()
else:
    print("\nSkipping random prediction as test_paths or test_labels are not available.")



plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_initial.history['accuracy'] + history_finetune.history['accuracy'])
plt.plot(history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy'])
plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')


plt.subplot(1, 2, 2)
plt.plot(history_initial.history['loss'] + history_finetune.history['loss'])
plt.plot(history_initial.history['val_loss'] + history_finetune.history['val_loss'])
plt.title('Combined Model Loss (Initial + Fine-tuning)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

#ResNet101 + DA

In [ ]:

from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras



IMG_SIZE = 224


base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


print("Freezing base ResNet101 layers...")
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()


epochs_initial = 20
print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")
history_initial = model.fit(
    train_ds,
    epochs=epochs_initial,
    validation_data=val_ds,
    #callbacks=[early_stopping, lr_reduction]
)


print("\nUnfreezing layers for fine-tuning...")


for layer in base_model.layers:
    layer.trainable = True

num_layers_to_unfreeze = 100


for layer in base_model.layers[:-num_layers_to_unfreeze]:
    layer.trainable = False
print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")



optimizer_finetune = Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model recompiled for fine-tuning with a very low learning rate.")
model.summary()

epochs_finetune = 50
print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")


history_finetune = model.fit(
    train_ds,
    epochs=epochs_finetune,
    validation_data=val_ds,
    #callbacks=[early_stopping, lr_reduction]
)


print("\nEvaluating the fine-tuned ResNet101 model on the test set...")
loss_finetune, accuracy_finetune = model.evaluate(test_data)
print(f"Fine-tuned Test Loss: {loss_finetune}")
print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


print("\nClassification Report for Fine-tuned ResNet101:")

y_true = test_labels_encoded


y_pred_finetune = model.predict(test_data)
y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)
report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
print(report_finetune)


model.save('resnet101_finetuned_model.h5')
print("Fine-tuned ResNet101 model saved successfully!")


if 'test_paths' in globals() and 'test_labels' in globals():
    random_index = np.random.randint(0, len(test_paths))
    image_path = test_paths[random_index]
    true_label = test_labels[random_index]


    img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)


    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


    plt.imshow(img)
    plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    plt.axis('off')
    plt.show()
else:
    print("\nSkipping random prediction as test_paths or test_labels are not available.")


plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
combined_accuracy = history_initial.history['accuracy'] + history_finetune.history['accuracy']
combined_val_accuracy = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
plt.plot(combined_accuracy)
plt.plot(combined_val_accuracy)
plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
combined_loss = history_initial.history['loss'] + history_finetune.history['loss']
combined_val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
plt.plot(combined_loss)
plt.plot(combined_val_loss)
plt.title('Combined Model Loss (Initial + Fine-tuning)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


#EfficientNetV2B0 + DA

In [ ]:

from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.preprocessing.image import ImageDataGenerator


IMG_SIZE = 224


base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))


print("Freezing base EfficientNetV2B0 layers for initial training...")
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

num_classes = train_labels_categorical.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()

epochs_initial = 20
print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")

train_data_to_use = train_ds if 'augmented_train_data' in globals() and augmented_train_data.samples > 0 else train_data

history_initial = model.fit(
    train_data_to_use,
    epochs=epochs_initial,
    validation_data=val_ds,
    #callbacks=[early_stopping, lr_reduction]
)


print("\nUnfreezing layers for fine-tuning...")


for layer in base_model.layers:
    layer.trainable = True


num_layers_to_unfreeze = 50

for layer in base_model.layers[:-num_layers_to_unfreeze]:
    layer.trainable = False
print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")


optimizer_finetune = Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model recompiled for fine-tuning with a very low learning rate.")
model.summary()


epochs_finetune = 50
print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")



history_finetune = model.fit(
    train_data_to_use,
    epochs=epochs_finetune,
    validation_data=val_ds,
    #callbacks=[early_stopping, lr_reduction]
)

print("\nEvaluating the fine-tuned EfficientNetV2B0 model on the test set...")

loss_finetune, accuracy_finetune = model.evaluate(test_data)
print(f"Fine-tuned Test Loss: {loss_finetune}")
print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


print("\nClassification Report for Fine-tuned EfficientNetV2B0:")

y_true = test_labels_encoded


y_pred_finetune = model.predict(test_data)
y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)
report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
print(report_finetune)


model.save('efficientnetv2b0_finetuned_model.h5')
print("Fine-tuned EfficientNetV2B0 model saved successfully!")


if 'test_paths' in globals() and 'test_labels' in globals():
    random_index = np.random.randint(0, len(test_paths))
    image_path = test_paths[random_index]
    true_label = test_labels[random_index]


    img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = keras.applications.efficientnet_v2.preprocess_input(img_array)


    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


    plt.imshow(img)
    plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    plt.axis('off')
    plt.show()
else:
    print("\nSkipping random prediction as test_paths or test_labels are not available.")


plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)

combined_accuracy = history_initial.history['accuracy'] + history_finetune.history['accuracy']
combined_val_accuracy = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
plt.plot(combined_accuracy)
plt.plot(combined_val_accuracy)
plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')


plt.subplot(1, 2, 2)
combined_loss = history_initial.history['loss'] + history_finetune.history['loss']
combined_val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
plt.plot(combined_loss)
plt.plot(combined_val_loss)
plt.title('Combined Model Loss (Initial + Fine-tuning)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


#EfficientNetV2S + DA

In [ ]:

from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os


IMG_SIZE = 224


print(f"Loading EfficientNetV2S model with input shape ({IMG_SIZE}, {IMG_SIZE}, 3)...")
base_model = EfficientNetV2S(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

print("Freezing base EfficientNetV2S layers for initial training...")
for layer in base_model.layers:
  layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

if 'train_labels_categorical' not in globals():
    print("Error: 'train_labels_categorical' is not defined. Cannot determine number of classes.")

    num_classes = 4
    print(f"Assuming {num_classes} classes.")
else:
    num_classes = train_labels_categorical.shape[1]
    print(f"Detected {num_classes} classes from train_labels_categorical.")

predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=7, factor=0.5, verbose=1)


train_data_to_use = train_ds if 'augmented_train_data' in globals() and augmented_train_data.samples > 0 else train_data

if 'train_data' not in globals() and ('augmented_train_data' not in globals() or augmented_train_data.samples == 0):
     print("Error: Training data generator ('train_data' or 'augmented_train_data') not found or is empty.")
else:

    epochs_initial = 20
    print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")


    if 'val_data' not in globals():
         print("Error: Validation data generator ('val_data') not found.")
    else:
        history_initial = model.fit(
            train_data_to_use,
            epochs=epochs_initial,
            validation_data=val_ds,
            #callbacks=[early_stopping, lr_reduction]
        )


        print("\nUnfreezing layers for fine-tuning...")


        for layer in base_model.layers:
            layer.trainable = True


        num_layers_to_unfreeze = 50


        for layer in base_model.layers[:-num_layers_to_unfreeze]:
            layer.trainable = False
        print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")


        optimizer_finetune = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

        print("Model recompiled for fine-tuning with a very low learning rate.")
        model.summary()


        epochs_finetune = 50
        print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")


        history_finetune = model.fit(
            train_data_to_use,
            epochs=epochs_finetune,
            validation_data=val_ds,

        )


        print("\nEvaluating the fine-tuned EfficientNetV2S model on the test set...")

        if 'test_data' not in globals():
            print("Error: Test data generator ('test_data') not found.")
        else:
            loss_finetune, accuracy_finetune = model.evaluate(test_data)
            print(f"Fine-tuned Test Loss: {loss_finetune}")
            print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


            print("\nClassification Report for Fine-tuned EfficientNetV2S:")


            if 'test_labels_encoded' in globals() and 'label_encoder' in globals():

                y_true = test_labels_encoded


                y_pred_finetune = model.predict(test_data)
                y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)

                report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
                print(report_finetune)
            else:
                 print("Error: 'test_labels_encoded' or 'label_encoder' not found. Cannot generate classification report.")


            model.save('efficientnetv2s_finetuned_model.h5')
            print("Fine-tuned EfficientNetV2S model saved successfully!")


            if 'test_paths' in globals() and 'test_labels' in globals() and 'label_encoder' in globals():
                random_index = np.random.randint(0, len(test_paths))
                image_path = test_paths[random_index]
                true_label = test_labels[random_index]


                img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = keras.preprocessing.image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)

                img_array = keras.applications.efficientnet_v2.preprocess_input(img_array)


                prediction = model.predict(img_array)
                predicted_class_index = np.argmax(prediction)

                predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


                plt.imshow(img)
                plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
                plt.axis('off')
                plt.show()
            else:
                print("\nSkipping random prediction as test_paths, test_labels, or label_encoder are not available.")



            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)

            combined_accuracy = history_initial.history['accuracy'] + history_finetune.history['accuracy']
            combined_val_accuracy = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
            plt.plot(combined_accuracy)
            plt.plot(combined_val_accuracy)
            plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')


            plt.subplot(1, 2, 2)
            combined_loss = history_initial.history['loss'] + history_finetune.history['loss']
            combined_val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
            plt.plot(combined_loss)
            plt.plot(combined_val_loss)
            plt.title('Combined Model Loss (Initial + Fine-tuning)')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')

            plt.tight_layout()
            plt.show()



# EficientNetB7 + DA

In [ ]:

from tensorflow.keras.applications import EfficientNetB7


IMG_SIZE = 224


print(f"Loading EfficientNetB7 model with input shape ({IMG_SIZE}, {IMG_SIZE}, 3)...")
base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

print("Freezing base EfficientNetB7 layers for initial training...")
for layer in base_model.layers:
  layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)



if 'train_labels_categorical' not in globals():
    print("Error: 'train_labels_categorical' is not defined. Cannot determine number of classes.")

    num_classes = 4
    print(f"Assuming {num_classes} classes.")
else:
    num_classes = train_labels_categorical.shape[1]
    print(f"Detected {num_classes} classes from train_labels_categorical.")

predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=7, factor=0.5, verbose=1)


train_data_to_use = train_ds if 'augmented_train_data' in globals() and augmented_train_data.samples > 0 else train_data

if 'train_data' not in globals() and ('augmented_train_data' not in globals() or augmented_train_data.samples == 0):
     print("Error: Training data generator ('train_data' or 'augmented_train_data') not found or is empty.")
else:

    epochs_initial = 20
    print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")


    if 'val_data' not in globals():
         print("Error: Validation data generator ('val_data') not found.")
    else:
        history_initial = model.fit(
            train_data_to_use,
            epochs=epochs_initial,
            validation_data=val_ds,
            #callbacks=[early_stopping, lr_reduction]
        )


        print("\nUnfreezing layers for fine-tuning...")


        for layer in base_model.layers:
            layer.trainable = True


        num_layers_to_unfreeze = 50


        for layer in base_model.layers[:-num_layers_to_unfreeze]:
            layer.trainable = False
        print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")


        optimizer_finetune = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

        print("Model recompiled for fine-tuning with a very low learning rate.")
        model.summary()


        epochs_finetune = 50
        print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")


        history_finetune = model.fit(
            train_data_to_use,
            epochs=epochs_finetune,
            validation_data=val_ds,
            #callbacks=[early_stopping, lr_reduction]
        )


        print("\nEvaluating the fine-tuned EfficientNetB7 model on the test set...")

        if 'test_data' not in globals():
            print("Error: Test data generator ('test_data') not found.")
        else:
            loss_finetune, accuracy_finetune = model.evaluate(test_data)
            print(f"Fine-tuned Test Loss: {loss_finetune}")
            print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


            print("\nClassification Report for Fine-tuned EfficientNetB7:")


            if 'test_labels_encoded' in globals() and 'label_encoder' in globals():

                y_true = test_labels_encoded


                y_pred_finetune = model.predict(test_data)
                y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)

                report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
                print(report_finetune)
            else:
                 print("Error: 'test_labels_encoded' or 'label_encoder' not found. Cannot generate classification report.")


            model.save('efficientnetb7_finetuned_model.h5')
            print("Fine-tuned EfficientNetB7 model saved successfully!")


            if 'test_paths' in globals() and 'test_labels' in globals() and 'label_encoder' in globals():
                random_index = np.random.randint(0, len(test_paths))
                image_path = test_paths[random_index]
                true_label = test_labels[random_index]


                img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = keras.preprocessing.image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)

                try:

                    from keras_applications.efficientnet import preprocess_input as efficientnet_preprocess_input
                    img_array = efficientnet_preprocess_input(img_array)
                except ImportError:
                    print("Warning: keras_applications or tensorflow_addons not found. Using generic preprocessing.")

                    img_array /= 255.0



                prediction = model.predict(img_array)
                predicted_class_index = np.argmax(prediction)
                predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


                plt.imshow(img)
                plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
                plt.axis('off')
                plt.show()
            else:
                print("\nSkipping random prediction as test_paths, test_labels, or label_encoder are not available.")



            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)

            combined_accuracy = history_initial.history['accuracy'] + history_finetune.history['accuracy']
            combined_val_accuracy = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
            plt.plot(combined_accuracy)
            plt.plot(combined_val_accuracy)
            plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')


            plt.subplot(1, 2, 2)
            combined_loss = history_initial.history['loss'] + history_finetune.history['loss']
            combined_val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
            plt.plot(combined_loss)
            plt.plot(combined_val_loss)
            plt.title('Combined Model Loss (Initial + Fine-tuning)')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')

            plt.tight_layout()
            plt.show()


#MobileNetV2 + DA

In [ ]:

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess_input



IMG_SIZE = 224


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


print("Freezing base MobileNetV2 layers...")
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)


if 'train_labels_categorical' not in globals():
    print("Error: 'train_labels_categorical' is not defined. Cannot determine number of classes.")
    num_classes = 4
    print(f"Assuming {num_classes} classes.")
else:
    num_classes = train_labels_categorical.shape[1]
    print(f"Detected {num_classes} classes from train_labels_categorical.")

predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)


train_data_to_use = train_ds if 'augmented_train_data' in globals() and augmented_train_data.samples > 0 else train_data

if 'train_data' not in globals() and ('augmented_train_data' not in globals() or augmented_train_data.samples == 0):
     print("Error: Training data generator ('train_data' or 'augmented_train_data') not found or is empty.")
else:

    epochs_initial = 20
    print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")


    if 'val_data' not in globals():
         print("Error: Validation data generator ('val_data') not found.")
    else:
        history_initial = model.fit(
            train_data_to_use,
            epochs=epochs_initial,
            validation_data=val_ds,

        )


        print("\nUnfreezing layers for fine-tuning...")


        for layer in base_model.layers:
            layer.trainable = True


        num_layers_to_unfreeze = 30


        for layer in base_model.layers[:-num_layers_to_unfreeze]:
            layer.trainable = False
        print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")


        optimizer_finetune = Adam(learning_rate=0.0001)
        model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

        print("Model recompiled for fine-tuning with a lower learning rate.")
        model.summary()


        epochs_finetune = 50
        print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")

        early_stopping_ft = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
        lr_reduction_ft = ReduceLROnPlateau(monitor='val_loss', patience=7, factor=0.5, verbose=1)


        history_finetune = model.fit(
            train_data_to_use,
            epochs=epochs_finetune,
            validation_data=val_ds,
            #callbacks=[early_stopping_ft, lr_reduction_ft]
        )


        print("\nEvaluating the fine-tuned MobileNetV2 model on the test set...")

        if 'test_data' not in globals():
            print("Error: Test data generator ('test_data') not found.")
        else:
            loss_finetune, accuracy_finetune = model.evaluate(test_data)
            print(f"Fine-tuned Test Loss: {loss_finetune}")
            print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")

            print("\nClassification Report for Fine-tuned MobileNetV2:")


            if 'test_labels_encoded' in globals() and 'label_encoder' in globals():

                y_true = test_labels_encoded


                y_pred_finetune = model.predict(test_data)
                y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)

                report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
                print(report_finetune)
            else:
                 print("Error: 'test_labels_encoded' or 'label_encoder' not found. Cannot generate classification report.")


            model.save('mobilenetv2_finetuned_model.h5')
            print("Fine-tuned MobileNetV2 model saved successfully!")


            if 'test_paths' in globals() and 'test_labels' in globals() and 'label_encoder' in globals():
                random_index = np.random.randint(0, len(test_paths))
                image_path = test_paths[random_index]
                true_label = test_labels[random_index]


                img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = keras.preprocessing.image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)

                img_array = mobilenetv2_preprocess_input(img_array)



                prediction = model.predict(img_array)
                predicted_class_index = np.argmax(prediction)

                predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


                plt.imshow(img)
                plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
                plt.axis('off')
                plt.show()
            else:
                print("\nSkipping random prediction as test_paths, test_labels, or label_encoder are not available.")



            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)

            combined_accuracy = history_initial.history['accuracy'] + history_finetune.history['accuracy']
            combined_val_accuracy = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
            plt.plot(combined_accuracy)
            plt.plot(combined_val_accuracy)
            plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')


            plt.subplot(1, 2, 2)
            combined_loss = history_initial.history['loss'] + history_finetune.history['loss']
            combined_val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
            plt.plot(combined_loss)
            plt.plot(combined_val_loss)
            plt.title('Combined Model Loss (Initial + Fine-tuning)')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')

            plt.tight_layout()
            plt.show()


# VGG19 + DA

In [ ]:

from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg19 import preprocess_input

IMG_SIZE = 224


print(f"Loading VGG19 model with input shape ({IMG_SIZE}, {IMG_SIZE}, 3)...")
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


print("Freezing base VGG19 layers for initial training...")
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


if 'train_labels_categorical' not in globals():
    print("Error: 'train_labels_categorical' is not defined. Cannot determine number of classes.")
    num_classes = 4
    print(f"Assuming {num_classes} classes.")
else:
    num_classes = train_labels_categorical.shape[1]
    print(f"Detected {num_classes} classes from train_labels_categorical.")

predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=7, factor=0.5, verbose=1)


train_data_to_use = train_ds if 'augmented_train_data' in globals() and augmented_train_data.samples > 0 else train_data

if 'train_data' not in globals() and ('augmented_train_data' not in globals() or augmented_train_data.samples == 0):
     print("Error: Training data generator ('train_data' or 'augmented_train_data') not found or is empty.")
else:

    epochs_initial = 20
    print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")


    if 'val_data' not in globals():
         print("Error: Validation data generator ('val_data') not found.")
    else:
        history_initial = model.fit(
            train_data_to_use,
            epochs=epochs_initial,
            validation_data=val_ds,
            #callbacks=[early_stopping, lr_reduction]
        )


        print("\nUnfreezing layers for fine-tuning...")


        for layer in base_model.layers:
            layer.trainable = True


        num_layers_to_unfreeze = 30


        for layer in base_model.layers[:-num_layers_to_unfreeze]:
            layer.trainable = False
        print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")


        optimizer_finetune = Adam(learning_rate=0.00001)
        model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

        print("Model recompiled for fine-tuning with a very low learning rate.")
        model.summary()


        epochs_finetune = 50
        print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")


        early_stopping_ft = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lr_reduction_ft = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)


        history_finetune = model.fit(
            train_data_to_use,
            epochs=epochs_finetune,
            validation_data=val_ds,
            #callbacks=[early_stopping_ft, lr_reduction_ft] # Use callbacks
        )


        print("\nEvaluating the fine-tuned VGG19 model on the test set...")

        if 'test_data' not in globals():
            print("Error: Test data generator ('test_data') not found.")
        else:
            loss_finetune, accuracy_finetune = model.evaluate(test_data)
            print(f"Fine-tuned Test Loss: {loss_finetune}")
            print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


            print("\nClassification Report for Fine-tuned VGG19:")


            if 'test_labels_encoded' in globals() and 'label_encoder' in globals():

                y_true = test_labels_encoded


                y_pred_finetune = model.predict(test_data)
                y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)

                report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
                print(report_finetune)
            else:
                 print("Error: 'test_labels_encoded' or 'label_encoder' not found. Cannot generate classification report.")


            model.save('vgg19_finetuned_model.h5')
            print("Fine-tuned VGG19 model saved successfully!")


            if 'test_paths' in globals() and 'test_labels' in globals() and 'label_encoder' in globals():
                random_index = np.random.randint(0, len(test_paths))
                image_path = test_paths[random_index]
                true_label = test_labels[random_index]


                img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = keras.preprocessing.image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = preprocess_input(img_array)



                prediction = model.predict(img_array)
                predicted_class_index = np.argmax(prediction)

                predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


                plt.imshow(img)
                plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
                plt.axis('off')
                plt.show()
            else:
                print("\nSkipping random prediction as test_paths, test_labels, or label_encoder are not available.")



            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)

            combined_accuracy = history_initial.history['accuracy'] + history_finetune.history['accuracy']
            combined_val_accuracy = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
            plt.plot(combined_accuracy)
            plt.plot(combined_val_accuracy)
            plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')


            plt.subplot(1, 2, 2)
            combined_loss = history_initial.history['loss'] + history_finetune.history['loss']
            combined_val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
            plt.plot(combined_loss)
            plt.plot(combined_val_loss)
            plt.title('Combined Model Loss (Initial + Fine-tuning)')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')

            plt.tight_layout()
            plt.show()


# VGG16 + DA

In [ ]:

from tensorflow.keras.applications import VGG16


IMG_SIZE = 224


print(f"Loading VGG16 model with input shape ({IMG_SIZE}, {IMG_SIZE}, 3)...")
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


print("Freezing base VGG16 layers for initial training...")
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)


if 'train_labels_categorical' not in globals():
    print("Error: 'train_labels_categorical' is not defined. Cannot determine number of classes.")

    num_classes = 4
    print(f"Assuming {num_classes} classes.")
else:
    num_classes = train_labels_categorical.shape[1]
    print(f"Detected {num_classes} classes from train_labels_categorical.")

predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("Model compiled for initial training (base layers frozen).")
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=7, factor=0.5, verbose=1)


train_data_to_use = train_ds if 'augmented_train_data' in globals() and augmented_train_data.samples > 0 else train_data

if 'train_data' not in globals() and ('augmented_train_data' not in globals() or augmented_train_data.samples == 0):
     print("Error: Training data generator ('train_data' or 'augmented_train_data') not found or is empty.")
else:

    epochs_initial = 20
    print(f"\nStarting initial training for {epochs_initial} epochs (base layers frozen)...")


    if 'val_data' not in globals():
         print("Error: Validation data generator ('val_data') not found.")
    else:
        history_initial = model.fit(
            train_data_to_use,
            epochs=epochs_initial,
            validation_data=val_ds,
            #callbacks=[early_stopping, lr_reduction]
        )


        print("\nUnfreezing layers for fine-tuning...")


        for layer in base_model.layers:
            layer.trainable = True


        num_layers_to_unfreeze = 30


        for layer in base_model.layers[:-num_layers_to_unfreeze]:
            layer.trainable = False
        print(f"Unfrozen {num_layers_to_unfreeze} layers in the base model for fine-tuning.")


        optimizer_finetune = Adam(learning_rate=0.00001)
        model.compile(optimizer=optimizer_finetune, loss='categorical_crossentropy', metrics=['accuracy'])

        print("Model recompiled for fine-tuning with a very low learning rate.")
        model.summary()


        epochs_finetune = 50
        print(f"\nStarting fine-tuning for {epochs_finetune} epochs...")


        early_stopping_ft = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lr_reduction_ft = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)


        history_finetune = model.fit(
            train_data_to_use,
            epochs=epochs_finetune,
            validation_data=val_data,
            #callbacks=[early_stopping_ft, lr_reduction_ft]
        )


        print("\nEvaluating the fine-tuned VGG16 model on the test set...")

        if 'test_data' not in globals():
            print("Error: Test data generator ('test_data') not found.")
        else:
            loss_finetune, accuracy_finetune = model.evaluate(test_data)
            print(f"Fine-tuned Test Loss: {loss_finetune}")
            print(f"Fine-tuned Test Accuracy: {accuracy_finetune}")


            print("\nClassification Report for Fine-tuned VGG16:")


            if 'test_labels_encoded' in globals() and 'label_encoder' in globals():

                y_true = test_labels_encoded


                y_pred_finetune = model.predict(test_data)
                y_pred_classes_finetune = np.argmax(y_pred_finetune, axis=1)

                report_finetune = classification_report(y_true, y_pred_classes_finetune, target_names=label_encoder.classes_)
                print(report_finetune)
            else:
                 print("Error: 'test_labels_encoded' or 'label_encoder' not found. Cannot generate classification report.")


            model.save('vgg16_finetuned_model.h5')
            print("Fine-tuned VGG16 model saved successfully!")


            if 'test_paths' in globals() and 'test_labels' in globals() and 'label_encoder' in globals():
                random_index = np.random.randint(0, len(test_paths))
                image_path = test_paths[random_index]
                true_label = test_labels[random_index]


                img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = keras.preprocessing.image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)

                img_array = preprocess_input(img_array)



                prediction = model.predict(img_array)
                predicted_class_index = np.argmax(prediction)

                predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]


                plt.imshow(img)
                plt.title(f"True Label: {true_label}, Predicted Label: {predicted_label}")
                plt.axis('off')
                plt.show()
            else:
                print("\nSkipping random prediction as test_paths, test_labels, or label_encoder are not available.")



            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)

            combined_accuracy = history_initial.history['accuracy'] + history_finetune.history['accuracy']
            combined_val_accuracy = history_initial.history['val_accuracy'] + history_finetune.history['val_accuracy']
            plt.plot(combined_accuracy)
            plt.plot(combined_val_accuracy)
            plt.title('Combined Model Accuracy (Initial + Fine-tuning)')
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')


            plt.subplot(1, 2, 2)
            combined_loss = history_initial.history['loss'] + history_finetune.history['loss']
            combined_val_loss = history_initial.history['val_loss'] + history_finetune.history['val_loss']
            plt.plot(combined_loss)
            plt.plot(combined_val_loss)
            plt.title('Combined Model Loss (Initial + Fine-tuning)')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')

            plt.tight_layout()
            plt.show()


In [ ]:
from tensorflow import keras
from tensorflow.keras.models import load_model


#Vision Transformer

In [ ]:
# STEP 1: Load model and freeze all but classifier
model = timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=len(class_names))
model.to(device)

# Freeze all ViT backbone layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classification head
for param in model.head.parameters():
    param.requires_grad = True

# Optimizer only for head
optimizer = optim.Adam(model.head.parameters(), lr=1e-3, weight_decay=1e-5)

criterion = nn.CrossEntropyLoss()
epochs_head = 5  # Train head for a few epochs

print("\n🔧 Training classification head only...\n")
for epoch in range(epochs_head):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs_head} - Head Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Head Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f} | Acc={100*correct/total:.2f}%")

# STEP 2: Unfreeze all layers for fine-tuning
for param in model.parameters():
    param.requires_grad = True

# New optimizer for full model with smaller LR
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
epochs_finetune = 20

print("\n🎯 Fine-tuning the entire model...\n")
best_val_acc = 0.0
for epoch in range(epochs_finetune):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs_finetune} - Fine-tuning"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss = val_loss / len(val_loader)

    print(f"Finetune Epoch {epoch+1}: Train Acc={train_accuracy:.2f}% | Val Acc={val_accuracy:.2f}%")

    # Save best model
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        best_model_state = model.state_dict()
        torch.save(model.state_dict(), "best_finetuned_vit.pth")
        print("✅ Best model saved.")

print(f"\n🏁 Best Validation Accuracy: {best_val_acc:.2f}%")


In [ ]:

model.eval()

train_correct = 0
train_total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

train_accuracy = 100 * train_correct / train_total
print(f"Training Accuracy: {train_accuracy:.2f}%")


y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.cpu().numpy())

test_accuracy = 100 * np.sum(np.array(y_true) == np.array(y_pred)) / len(y_true)
print(f"Testing Accuracy: {test_accuracy:.2f}%")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# **X-RVV with Data Augmentation**

In [ ]:
import tensorflow as tf
from keras.applications import ResNet50, VGG19
from keras.layers import Input, Dense, Dropout, concatenate, GlobalAveragePooling2D, Resizing, Rescaling
from keras.models import Model
from keras.optimizers import Adam
from transformers import TFViTModel
import segmentation_models as sm


vit_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

IMG_SIZE = 224
NUM_CLASSES = 4


input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input_layer')


resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)
resnet_out = GlobalAveragePooling2D()(resnet.output)


vgg = VGG19(include_top=False, weights='imagenet', input_tensor=input_tensor)
vgg_out = GlobalAveragePooling2D()(vgg.output)


vit_input = Resizing(224, 224)(input_tensor)
vit_input = Rescaling(1./255)(vit_input)

class TransposeLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.transpose(inputs, perm=[0, 3, 1, 2])

vit_input_transposed = TransposeLayer()(vit_input)

class ViTBranch(tf.keras.layers.Layer):
    def call(self, inputs, training=False):
        outputs = vit_model({'pixel_values': inputs})
        return outputs.last_hidden_state[:, 0, :]

vit_out = ViTBranch()(vit_input_transposed)


merged = concatenate([resnet_out, vgg_out, vit_out])
x = Dense(1024, activation='relu')(merged)
x = Dropout(0.6)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
classification_output = Dense(NUM_CLASSES, activation='softmax', name='classification_output')(x)

segmentation_output = sm.Unet(
    backbone_name='resnet34',
    encoder_weights='imagenet',
    classes=1,
    activation='sigmoid'
)(input_tensor)
segmentation_output._name = 'segmentation_output'

model = Model(inputs=input_tensor, outputs=[classification_output, segmentation_output], name="functional_37")


classification_model = Model(inputs=model.input, outputs=model.get_layer('classification_output').output, name="functional_37")


classification_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

classification_model.summary()


In [ ]:
history = classification_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[
        keras.callbacks.ModelCheckpoint('best_hybrid_model2.keras', save_best_only=True)
    ]
)

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
classification_model.save('best_hybrid_classification_model3.keras')

# Testing Hybrid model on Test Data

In [ ]:
test_loss, test_accuracy = classification_model.evaluate(test_ds)
print(f" Test Accuracy: {test_accuracy:.4f}")

In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = classification_model.predict(images)
    y_true.extend(tf.argmax(labels, axis=1).numpy())
    y_pred.extend(tf.argmax(preds, axis=1).numpy())

y_true = tf.convert_to_tensor(y_true)
y_pred = tf.convert_to_tensor(y_pred)

In [ ]:
from sklearn.metrics import classification_report

class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix - Test Set")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:

from sklearn.metrics import roc_auc_score, roc_curve, auc
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt



y_true_auc = []
y_prob = []


for images, labels in test_ds:

    probs = classification_model.predict(images, verbose=0)

    y_true_auc.extend(tf.argmax(labels, axis=1).numpy())

    y_prob.extend(probs)

y_true_auc = np.array(y_true_auc)
y_prob = np.array(y_prob)


auc_score_ovr = roc_auc_score(y_true_auc, y_prob, multi_class='ovr')


auc_score_macro = roc_auc_score(y_true_auc, y_prob, multi_class='ovr', average='macro')

auc_score_weighted = roc_auc_score(y_true_auc, y_prob, multi_class='ovr', average='weighted')


print(f"\nAUC Score (One-vs-Rest): {auc_score_ovr:.4f}")
print(f"AUC Score (Macro Average): {auc_score_macro:.4f}")
print(f"AUC Score (Weighted Average): {auc_score_weighted:.4f}")


print("\nAUC per Class:")
for i, class_name in enumerate(class_names):

    y_true_class = (y_true_auc == i).astype(int)

    y_prob_class = y_prob[:, i]

    auc_class = roc_auc_score(y_true_class, y_prob_class)
    print(f"AUC for class '{class_name}': {auc_class:.4f}")



fpr = dict()
tpr = dict()
roc_auc = dict()

n_classes = len(class_names)

for i in range(n_classes):

    y_true_bin = (y_true_auc == i).astype(int)

    fpr[i], tpr[i], _ = roc_curve(y_true_bin, y_prob[:, i])

    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Per Class (OvR)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


for i in range(n_classes):
    plt.figure(figsize=(6, 5))
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for Class: {class_names[i]}')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()


#GradCAM Implementation

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from transformers import TFViTModel
import keras


@keras.saving.register_keras_serializable()
class ViTBranch(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.vit = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

    def call(self, inputs, training=False):
        return self.vit({'pixel_values': inputs}, training=training).last_hidden_state[:, 0, :]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 768)


@keras.saving.register_keras_serializable()
class TransposeLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.transpose(inputs, perm=[0, 3, 1, 2])


model = tf.keras.models.load_model('best_hybrid_classification_model3.keras',
                                   custom_objects={'TransposeLayer': TransposeLayer, 'ViTBranch': ViTBranch})


def generate_gradcam(model, img, class_index, target_layer='conv5_block3_out'):
    """Generates Grad-CAM++ visualization for classification without segmentation."""
    img_resized = cv2.resize(img, (224, 224))
    x = tf.keras.applications.resnet50.preprocess_input(np.expand_dims(img_resized.copy(), axis=0))


    classification_output_layer = model.get_layer('classification_output')
    grad_model = tf.keras.models.Model([model.input],
                                       [model.get_layer(target_layer).output, classification_output_layer.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(x)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap + 1e-8)
    heatmap = tf.image.resize(heatmap[..., tf.newaxis], (224, 224), method='bilinear')
    heatmap = tf.squeeze(heatmap).numpy()


    heatmap = np.power(heatmap, 2.5)
    heatmap[heatmap < 0.4] = 0


    heatmap_uint8 = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)


    gray_img = cv2.cvtColor(img_resized.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    gray_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
    overlay = cv2.addWeighted(gray_img, 0.7, heatmap_color, 0.6, 0)

    return overlay, img_resized


def plot_top1_correct_gradcam(model, test_ds, class_names):
    """Plots Grad-CAM++ for top-1 correct prediction (classification only)."""
    correct_predictions = []
    count = 0

    for images, labels in test_ds:
        for i in range(len(images)):
            img = images[i].numpy()
            label = tf.argmax(labels[i]).numpy()

            preds = model.predict(np.expand_dims(img, axis=0))
            pred_class = np.argmax(preds[0])

            if pred_class == label:
                correct_predictions.append((img, label, pred_class))
                count += 1
                if count == 1:
                    break
        if count == 1:
            break

    for img, label, pred_class in correct_predictions:
        overlay, base_img = generate_gradcam(model, img, pred_class)

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(base_img, cv2.COLOR_RGB2GRAY), cmap='gray')
        plt.title(f"Original MRI\nTrue: {class_names[label]}")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
        plt.title(f"Grad-CAM++ Overlay\nPredicted: {class_names[pred_class]}")
        plt.axis("off")
        plt.tight_layout()
        plt.show()



plot_top1_correct_gradcam(model, test_ds, class_names)
